# Skin Data

In [1]:
import os.path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    _home = '/content/drive/MyDrive'
except ImportError:
    _home = '~'
finally:
    data_root = os.path.join(_home, '.pytorch')

#os.chdir(_home)
#root_path = '/content/drive/MyDrive'
print(data_root)

#!pwd

~\.pytorch


In [2]:
# necessary so that we don't run into problems while downloading data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd
import os
import time
from pathlib import Path
import torch
import torchvision
from torchvision import transforms
from torchvision import models
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import trange
from sklearn.utils import shuffle

device = 'cuda' if torch.cuda.is_available() else 'cpu'

C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\kroep\anaconda3\envs\standard_env\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
RANDOM_SEED = 42

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Define functions for loading and processing the image data

In [5]:
def get_CIFAR10_loader(root, train=True, batch_size=1, num_workers=0, shuffle=False):
    """
    Function to create dataloaders for CIFAR10 dataset.

    Args:
        root (str): Root directory of the dataset where dataset is saved.
        train (bool): If True, creates a dataloader for training set.
        batch_size (int): Number of samples per batch.
        num_workers (int): How many subprocesses to use for data loading.
        shuffle (bool): Set to True to have the data reshuffled at every epoch.

    Returns:
        train_loader, val_loader: Dataloaders for training and validation set.
    """

    # Define the image transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize([256]),
        transforms.CenterCrop([224]),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

    # Load the CIFAR10 dataset
    cifar_set = datasets.CIFAR10(root=root, train=True, download=True, transform=transform)

    # Split the dataset into training and validation sets (90% for training and 10% for validation)
    size_train = int(len(cifar_set) * 0.9)
    size_val = len(cifar_set) - size_train
    train_set, val_set = random_split(cifar_set, [size_train, size_val])
    
    # Create dataloaders for training and validation sets
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

    return train_loader, val_loader


def get_skin_loader(root, train=True, batch_size=1, num_workers=0, shuffle=False):
    """
    Function to create dataloaders for skin dataset.

    Args:
        root (str): Root directory of the dataset where dataset is saved.
        train (bool): If True, creates a dataloader for training set.
        batch_size (int): Number of samples per batch.
        num_workers (int): How many subprocesses to use for data loading.
        shuffle (bool): Set to True to have the data reshuffled at every epoch.

    Returns:
        train_loader, val_loader: Dataloaders for training and validation set.
    """

    # Load and shuffle the skin dataset
    df_skin = pd.read_csv(root)
    df_skin = df_skin.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    # Extract images from the dataframe and reshape them into the desired shape
    np_rgb = df_skin.drop('label', axis=1).to_numpy()
    images = np.reshape(np_rgb,(len(np_rgb), 28, 28, 3))

    # Reshape images from (28,28,3) to (3,28,28)
    images = np.moveaxis(images, -1, 1)

    # Normalize the images
    images = images/255

    # Combine images and labels into tuples
    image_labels_list = list(zip(images, df_skin['label'].to_numpy()))

    # Split the dataset into training and validation sets (90% for training and 10% for validation)
    size_train = int(len(images) * 0.9)
    size_val = len(images) - size_train
    train_set, val_set = random_split(image_labels_list, [size_train, size_val])

    # Create dataloaders for training and validation sets
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)

    return train_loader, val_loader


def process_image(img):
    """
    Function to process an individual image.

    Args:
        img (numpy.array): The image to process.

    Returns:
        downscaled_img.shape: The shape of the processed image.
    """

    # Upscale the image
    reshaped_img = np.repeat(np.repeat(img, repeats=8, axis=1), repeats=8, axis=2)

    # Convert the numpy array to a PIL image
    pil_img = Image.fromarray(reshaped_img.astype('uint8'), 'RGB')

    # Downscale the image back to its original size
    downscaled_img = torchvision.transforms.Resize((28,28))(pil_img)

    # Move the color dimension to the first position
    final_image = np.moveaxis(np.array(downscaled_img), 0, 2)

    return final_image.shape


# Load the skin dataset
df_skin = pd.read_csv('data\\hmnist_28_28_RGB.csv')

# Extract images from the dataframe and reshape them into the desired shape
np_rgb = df_skin.drop('label', axis=1).to_numpy()
images = np.reshape(np_rgb, (len(np_rgb), 28, 28, 3))

# Process the first image in the dataset
img = images[0]
print(process_image(img))

(28, 3, 28)


### Load Data and compare CIFAR10 to skin data

#### Skin data

In [6]:
# Load the skin dataset and split into training and validation sets
train_set, val_set = get_skin_loader("data\\hmnist_28_28_RGB.csv", train=True, batch_size=50, num_workers=2, shuffle=False)

# Define the classes (assumed from CIFAR10 dataset)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Get a batch of training images
dataiter = iter(train_set)
images, labels = next(dataiter)


def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    """
    Function to plot a batch of images.

    Args:
        imgs (list): List of images to plot.
        with_orig (bool): Whether to include the original image.
        row_title (str): Title for the row.
        **imshow_kwargs: Additional keyword arguments for imshow function.
    """
    # Check if the input is a 2D list
    if not isinstance(imgs[0], list):
        # If not, convert it into a 2D list
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)

    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    
    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()


class TransferCNN(nn.Module):
    def __init__(self, model_trained: nn.Module, num_classes: int = 10):
        """
        Initialize the new network.

        Args:
            model_trained (nn.Module): Pre-trained model.
            num_classes (int): Number of classes.
        """
        super(TransferCNN, self).__init__()
        self.num_classes = num_classes
        self.classifier = nn.Linear(1000, num_classes)
        self.model_trained = model_trained

    def forward(self, x):
        x = self.model_trained(x)
        x = self.classifier(x)
        return x

def _forward(network: nn.Module, data: DataLoader, metric: callable, give_logits=False, augment_data=False):
    """
    Define the forward pass helper function.

    Args:
        network (nn.Module): The neural network.
        data (DataLoader): The data loader.
        metric (callable): Metric to calculate.
        give_logits (bool): Whether to return logits.
        augment_data (bool): Whether to augment data.

    Returns:
        res: Results of forward pass.
    """
    device = next(network.parameters()).device
    
    transform = transforms.Compose([
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomResizedCrop(size=(28, 28), scale=(0.7, 1)),
    ])
    
    for x, y in data:
        if augment_data and np.random.uniform()>0.5:
            x = transform(x)
        x = np.repeat(np.repeat(x, repeats=8, axis=2), repeats=8, axis=3)
        x, y = x.to(device, dtype=torch.float), y.to(device, dtype=torch.long)
        logits = network(x)
        res = metric(logits, y)
        if give_logits:
            yield res, torch.argmax(logits.detach().clone(), dim=1).tolist(), y.detach().clone().tolist()
        else:
            yield res


def evaluate(network: nn.Module, data: DataLoader, metric: callable, give_logits=False) -> list:
    """
    Function to evaluate the model.

    Args:
        network (nn.Module): The model to evaluate.
        data (DataLoader): The data loader.
        metric (callable): Metric to evaluate.
        give_logits (bool): Whether to return logits.

    Returns:
        list: Evaluation results.
    """
    network.eval()
    if give_logits:
        yielded_results = _forward(network, data, metric, give_logits=True)        
        return [(res[0].item(), res[1], res[2]) for res in yielded_results]
    else:
        results = _forward(network, data, metric)
        return [res.item() for res in results]


def update(network: nn.Module, data: DataLoader, loss: nn.Module,
           opt: optim.Optimizer, augment_data=True) -> list:
    """
    Function to update the model parameters.

    Args:
        network (nn.Module): The model to update.
        data (DataLoader): The data loader.
        loss (nn.Module): The loss function.
        opt (optim.Optimizer): The optimizer.
        augment_data (bool): Whether to augment data.

    Returns:
        list: Errors for each batch.
    """
    network.train()
    
    errs = []
    for err in _forward(network, data, loss, augment_data=augment_data):
        errs.append(err.item())
        opt.zero_grad()
        err.backward()
        opt.step()
        
    return errs


def accuracy(logits, targets):
    """
    Function to compute the accuracy.

    Args:
        logits (Tensor): The logits from the model output.
        targets (Tensor): The true labels.

    Returns:
        Tensor: The accuracy.
    """
    top_pred = logits.argmax(1, keepdim = True)
    correct = top_pred.eq(targets.view_as(top_pred)).sum()
    acc = correct.float() / targets.shape[0]

    return acc

#### Shape of image as an input to the network has to be: ([3, 224, 224]). For now it should be ([3, 28, 28]), because we change the dimension function of the image in the function _forward().

#### Defined the function to generate data:

In [7]:
def generate_data(model, epochs:int, lr:float, use_augmented_data:bool, model_name:str,
                  train_set=train_set, val_set=val_set, device=device, to_folder='generated_df_data'):
    
    # Initialize a dataframe to store model details during training
    df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                    'confusion_matrix', 'train_errs', 'valid_errs', 'used_data_augmentation'])
    
    # Transfer model to appropriate device
    model.to(device)

    # Initialize optimizer and loss function
    optim = torch.optim.Adam(model.parameters(), lr = lr)
    ce = nn.CrossEntropyLoss()

    # Initialize lists and arrays to store various model parameters and metrics over epochs
    name_list, epoch_list, lr_list, conf_matrix_list, train_errs_list, valid_errs_list = [], [], [], [], [], []

    weights_shape = model.classifier.weight.shape
    bias_shape = model.classifier.bias.shape
    weights_array, bias_array = np.zeros((epochs,weights_shape[0],weights_shape[1])), np.zeros((epochs,bias_shape[0]))

    for i in trange(epochs):
        # Start a timer for each epoch
        start_time = time.time()

        # Perform training with or without data augmentation
        local_errs = update(model, train_set, ce, optim, augment_data=use_augmented_data)
        train_errs_list.append(sum(local_errs) / len(local_errs))

        # Perform validation and get output logits
        val_output = np.array(evaluate(model, val_set, ce, give_logits=True), dtype=object)
        val_errs = val_output[:,0]
        valid_errs_list.append(sum(val_errs) / len(val_errs))

        # Compute confusion matrix based on the true and predicted labels
        logits_argmax_list = val_output[:,1]
        y_list = val_output[:,2]
        conf_matrix = confusion_matrix(np.array(sum(y_list.tolist(), [])), np.array(sum(logits_argmax_list.tolist(), [])))
        conf_matrix_list.append(conf_matrix)

        # Compute training and validation accuracy
        acc_train = evaluate(model, train_set, metric = accuracy)
        acc_val = evaluate(model, val_set, metric = accuracy)

        name_list.append(model_name)
        epoch_list.append(i)
        lr_list.append(lr)

        # Store model parameters
        weights_array[i] = model.classifier.weight.detach().cpu().numpy()
        bias_array[i] = model.classifier.bias.detach().cpu().numpy()
        
        new_row = {'name': model_name, 
                   'epoch': i, 
                   'lr': lr, 
                   'classifier.weight': model.classifier.weight.detach().cpu().numpy(), 
                   'classifier.bias': model.classifier.bias.detach().cpu().numpy(), 
                   'confusion_matrix': conf_matrix, 
                   'train_errs': sum(local_errs) / len(local_errs), 
                   'valid_errs': sum(val_errs) / len(val_errs), 
                   'used_data_augmentation': use_augmented_data}

        df_data = df_data.append(new_row, ignore_index=True)
        
        # Save the current dataframe after each epoch
        df_data.to_pickle(f'{to_folder}\\{model_name}_Epochs{epochs}')
        
        # Print accuracy metrics for the current epoch
        print("Epoch {}: Accuracy on train data: {:.2f} %".format(i+1, sum(acc_train) / len(acc_train) * 100))
        print("Epoch {}: Accuracy on validation data: {:.2f} %".format(i+1, sum(acc_val) / len(acc_val) * 100))


        # Print time elapsed for current epoch
        print(f'         --- {round(time.time()-start_time, 2)} seconds ---')
    
    # Return final dataframe
    return df_data


In [8]:
def process_model(model_type: str, epochs: int, lr: float, use_augmented_data: bool, to_folder: str):
    """
    This function processes a pretrained model of the specified type, freezes its parameters, and generates data for it.

    Parameters:
    - model_type: Type of the model. Accepted values are 'resnet50' and 'alexnet'.
    - epochs: Number of epochs for data generation.
    - lr: Learning rate for data generation.
    - use_augmented_data: Whether to use augmented data or not.

    Returns:
    - A dataframe generated by the `generate_data` function for the specified model.
    """

    if model_type == 'resnet50':
        model_pretrained = models.resnet50(pretrained=True)
    elif model_type == 'alexnet':
        model_pretrained = models.alexnet(pretrained=True)
    else:
        raise ValueError("Invalid model type. Expected 'resnet50' or 'alexnet'.")
    
    for param in model_pretrained.parameters():
        param.requires_grad = False

    model_transfer_cnn = TransferCNN(model_pretrained, num_classes = 7)
    model_transfer_cnn.eval()
    
    augment_status = "DataAugmentation" if use_augmented_data else "NoDataAugmentation"
    model_name = f'{model_type}_LearningRate{str(lr)[2:]}_{augment_status}'

    df = generate_data(model_transfer_cnn, epochs=epochs, lr=lr, use_augmented_data=use_augmented_data,
                       model_name=model_name, to_folder=to_folder)

    return df

In [9]:
for model_type in ['alexnet', 'resnet50']:
    for lr in [1e-3, 1e-4]:
        epochs = 125 if lr==1e-4 else 75
        for use_augmented_data in [True, False]:
            print(f'\n{model_type}, lr={lr}, epochs={epochs}, use_augmented_data={use_augmented_data};')
            process_model(model_type=model_type, epochs=epochs, lr=lr,
                          use_augmented_data=use_augmented_data, to_folder='generated_df_data')


alexnet, lr=0.001, epochs=75, use_augmented_data=True;


  0%|          | 0/75 [00:00<?, ?it/s]

Epoch 1: Accuracy on train data: 68.82 %
Epoch 1: Accuracy on validation data: 69.81 %
         --- 55.58 seconds ---
Epoch 2: Accuracy on train data: 70.84 %
Epoch 2: Accuracy on validation data: 70.86 %
         --- 56.2 seconds ---
Epoch 3: Accuracy on train data: 69.17 %
Epoch 3: Accuracy on validation data: 69.05 %
         --- 57.2 seconds ---
Epoch 4: Accuracy on train data: 70.34 %
Epoch 4: Accuracy on validation data: 69.81 %
         --- 58.13 seconds ---
Epoch 5: Accuracy on train data: 69.59 %
Epoch 5: Accuracy on validation data: 68.19 %
         --- 58.76 seconds ---
Epoch 6: Accuracy on train data: 68.32 %
Epoch 6: Accuracy on validation data: 68.48 %
         --- 59.19 seconds ---
Epoch 7: Accuracy on train data: 71.50 %
Epoch 7: Accuracy on validation data: 70.48 %
         --- 58.8 seconds ---
Epoch 8: Accuracy on train data: 71.00 %
Epoch 8: Accuracy on validation data: 70.38 %
         --- 59.03 seconds ---
Epoch 9: Accuracy on train data: 62.77 %
Epoch 9: Accuracy 

Epoch 70: Accuracy on train data: 70.40 %
Epoch 70: Accuracy on validation data: 70.19 %
         --- 55.74 seconds ---
Epoch 71: Accuracy on train data: 72.74 %
Epoch 71: Accuracy on validation data: 71.81 %
         --- 55.7 seconds ---
Epoch 72: Accuracy on train data: 73.00 %
Epoch 72: Accuracy on validation data: 72.00 %
         --- 55.83 seconds ---
Epoch 73: Accuracy on train data: 71.16 %
Epoch 73: Accuracy on validation data: 71.05 %
         --- 56.53 seconds ---
Epoch 74: Accuracy on train data: 70.51 %
Epoch 74: Accuracy on validation data: 70.38 %
         --- 56.77 seconds ---
Epoch 75: Accuracy on train data: 66.06 %
Epoch 75: Accuracy on validation data: 65.71 %
         --- 56.55 seconds ---

alexnet, lr=0.001, epochs=75, use_augmented_data=False;


  0%|          | 0/75 [00:00<?, ?it/s]

...

Epoch 1: Accuracy on train data: 68.65 %
Epoch 1: Accuracy on validation data: 66.38 %
         --- 94.58 seconds ---
Epoch 2: Accuracy on train data: 70.69 %
Epoch 2: Accuracy on validation data: 70.86 %
         --- 94.79 seconds ---
Epoch 3: Accuracy on train data: 72.51 %
Epoch 3: Accuracy on validation data: 71.81 %
         --- 95.33 seconds ---
Epoch 4: Accuracy on train data: 73.16 %
Epoch 4: Accuracy on validation data: 72.57 %
         --- 94.98 seconds ---
Epoch 5: Accuracy on train data: 73.77 %
Epoch 5: Accuracy on validation data: 72.76 %
         --- 94.81 seconds ---
Epoch 6: Accuracy on train data: 74.08 %
Epoch 6: Accuracy on validation data: 73.05 %
         --- 95.05 seconds ---
Epoch 7: Accuracy on train data: 74.85 %
Epoch 7: Accuracy on validation data: 73.71 %
         --- 95.16 seconds ---
Epoch 8: Accuracy on train data: 75.29 %
Epoch 8: Accuracy on validation data: 74.48 %
         --- 94.88 seconds ---
Epoch 9: Accuracy on train data: 74.86 %
Epoch 9: Accura

Epoch 70: Accuracy on train data: 77.75 %
Epoch 70: Accuracy on validation data: 76.86 %
         --- 95.46 seconds ---
Epoch 71: Accuracy on train data: 77.37 %
Epoch 71: Accuracy on validation data: 76.48 %
         --- 95.59 seconds ---
Epoch 72: Accuracy on train data: 77.63 %
Epoch 72: Accuracy on validation data: 77.14 %
         --- 95.36 seconds ---
Epoch 73: Accuracy on train data: 77.52 %
Epoch 73: Accuracy on validation data: 75.71 %
         --- 95.91 seconds ---
Epoch 74: Accuracy on train data: 77.17 %
Epoch 74: Accuracy on validation data: 76.00 %
         --- 93.22 seconds ---
Epoch 75: Accuracy on train data: 77.43 %
Epoch 75: Accuracy on validation data: 76.67 %
         --- 93.82 seconds ---
Epoch 76: Accuracy on train data: 76.71 %
Epoch 76: Accuracy on validation data: 76.19 %
         --- 93.78 seconds ---
Epoch 77: Accuracy on train data: 77.59 %
Epoch 77: Accuracy on validation data: 75.43 %
         --- 93.96 seconds ---
Epoch 78: Accuracy on train data: 77.70 

  0%|          | 0/125 [00:00<?, ?it/s]

Epoch 1: Accuracy on train data: 68.71 %
Epoch 1: Accuracy on validation data: 68.00 %
         --- 93.92 seconds ---
Epoch 2: Accuracy on train data: 71.17 %
Epoch 2: Accuracy on validation data: 71.05 %
         --- 94.14 seconds ---
Epoch 3: Accuracy on train data: 72.91 %
Epoch 3: Accuracy on validation data: 72.67 %
         --- 94.07 seconds ---
Epoch 4: Accuracy on train data: 73.74 %
Epoch 4: Accuracy on validation data: 73.52 %
         --- 94.25 seconds ---
Epoch 5: Accuracy on train data: 74.51 %
Epoch 5: Accuracy on validation data: 73.52 %
         --- 94.08 seconds ---
Epoch 6: Accuracy on train data: 74.87 %
Epoch 6: Accuracy on validation data: 74.00 %
         --- 94.11 seconds ---
Epoch 7: Accuracy on train data: 75.20 %
Epoch 7: Accuracy on validation data: 73.81 %
         --- 94.29 seconds ---
Epoch 8: Accuracy on train data: 75.45 %
Epoch 8: Accuracy on validation data: 73.71 %
         --- 94.01 seconds ---
Epoch 9: Accuracy on train data: 75.76 %
Epoch 9: Accura

Epoch 70: Accuracy on train data: 78.55 %
Epoch 70: Accuracy on validation data: 76.38 %
         --- 95.84 seconds ---
Epoch 71: Accuracy on train data: 78.59 %
Epoch 71: Accuracy on validation data: 76.38 %
         --- 95.78 seconds ---
Epoch 72: Accuracy on train data: 78.60 %
Epoch 72: Accuracy on validation data: 76.48 %
         --- 95.99 seconds ---
Epoch 73: Accuracy on train data: 78.60 %
Epoch 73: Accuracy on validation data: 76.48 %
         --- 95.67 seconds ---
Epoch 74: Accuracy on train data: 78.62 %
Epoch 74: Accuracy on validation data: 76.48 %
         --- 95.38 seconds ---
Epoch 75: Accuracy on train data: 78.61 %
Epoch 75: Accuracy on validation data: 76.48 %
         --- 94.97 seconds ---
Epoch 76: Accuracy on train data: 78.62 %
Epoch 76: Accuracy on validation data: 76.48 %
         --- 95.27 seconds ---
Epoch 77: Accuracy on train data: 78.63 %
Epoch 77: Accuracy on validation data: 76.38 %
         --- 95.27 seconds ---
Epoch 78: Accuracy on train data: 78.66 

In [10]:
df_loaded = pd.read_pickle('generated_df_data\\alexnet_LearningRate001_DataAugmentation_Epochs75')
df_loaded.head()

,name,epoch,lr,classifier.weight,classifier.bias,confusion_matrix,train_errs,valid_errs,used_data_augmentation
0,alexnet_LearningRate001_DataAugmentation,0,0.001,"[[-0.013256238, -0.03859671, -0.01500524, -0.0...","[-0.028298642, -0.012024412, -0.021079782, 0.0...","[[3, 0, 9, 0, 13, 0, 7], [1, 0, 15, 0, 26, 0, ...",1.044060,0.846689,True
1,alexnet_LearningRate001_DataAugmentation,1,0.001,"[[-0.015944963, -0.04880148, -0.014346109, -0....","[-0.030114282, -0.012192469, -0.021237561, 0.0...","[[1, 2, 17, 0, 12, 0, 0], [0, 3, 23, 0, 21, 0,...",0.956414,0.816307,True
2,alexnet_LearningRate001_DataAugmentation,2,0.001,"[[-0.016647134, -0.049048353, -0.013311493, -0...","[-0.031581227, -0.012979197, -0.021635009, 0.0...","[[1, 0, 20, 0, 5, 0, 6], [0, 1, 29, 0, 13, 1, ...",0.949029,0.856132,True
3,alexnet_LearningRate001_DataAugmentation,3,0.001,"[[-0.01728396, -0.0510413, -0.012470494, -0.01...","[-0.032782834, -0.014987338, -0.01900003, 0.02...","[[2, 1, 17, 0, 7, 0, 5], [1, 6, 23, 0, 15, 0, ...",0.932115,0.819701,True
4,alexnet_LearningRate001_DataAugmentation,4,0.001,"[[-0.018036347, -0.040113434, -0.012168371, -0...","[-0.031929113, -0.019071028, -0.018729134, 0.0...","[[2, 0, 18, 0, 6, 0, 6], [1, 2, 28, 0, 11, 1, ...",0.941694,0.839014,True
